# Persistencia de modelos

Ya tienes todo listo, estás contento o contenta con tu modelo y todos los vectorizadores y transformadores que entrenaste para hacerlo funcionar. Pero, ¿qué sigue? es hora de guardarlo en el disco para distribuirlo y ponerlo en producción.

Para continuar en esta lección, voy a cargar y crear y entrenar un pipeline de scikit learn que para fines prácticos, es un modelo de scikit-learn. Si quieres conocer más detalles, revisa la lección sobre pipelines.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def load_trained_model():
# Generate synthetic data
	np.random.seed(0)
	X = np.random.rand(100, 1)
	y = 2 * X + 1 + 0.1 * np.random.randn(100, 1)
	
	# Split the data into training and testing sets
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
	
	# Create a linear regression model
	model = LinearRegression()
	
	# Train the model using the training data
	model.fit(X_train, y_train)

	return model

model = load_trained_model()

Recuerda que este ya es un modelo entrenado.

## Pickle

Tradicionalmente los modelos eran serializados en disco utilizando <code>pickle</code> la biblioteca por default para serializar objetos en Python.

Guardar un modelo es sencillo con pickle, lo que tienes que hacer es:

In [ ]:
import pickle

with open("model.pickle", "wb") as wb:
	pickle.dump(model, wb)

Esto serializará el modelo en el archivo “model.pickle”, este ya es un archivo que podemos compartir con alguien más o poner en producción nosotros mismos. Para leerlo de disco es necesario hacer lo siguiente:

In [ ]:
with open("model.pickle", "rb") as rb:
	unpickled_model = pickle.load(rb)

Podemos corroborar que sea del tipo que estamos esperando:

In [ ]:
type(unpickled_model)

Y hacer predicciones sobre nuevos datos:

### Desventajas

Sin embargo, pickle tiene serios fallos de seguridad y no es el modo recomendable de persistir tus modelos ya que al momento de deserializar este permite la ejecución de código de forma arbitraria. Yo te recomiendo que uses pickle si, y solo si, no tienes otra alternativa.

## Joblib

Existe otra biblioteca llamada <code>joblib</code> que está optimizada para serializar grandes arreglos de NumPy. Pero que internamente termina utilizando pickle para muchas de sus tareas, así que comparten los mismos problemas.

Yo te voy a mostrar cómo se usa por completitud en este curso:

In [ ]:
from joblib import dump

dump(model, "model.joblib")

Y para cargarlo, se hace así:

In [ ]:
from joblib import load

model_joblibbed = load("model.joblib")

Revisa el tipo y haz algunas predicciones para que veas que si funciona:

In [ ]:
type(model_joblibbed)

Si bien Joblib es una mejora sobre Pickle, esta mejora es en aspectos de velocidad y tamaño del modelo almacenado, pero nada cambia en el ámbito de seguridad. Es por eso que te recomiendo que no utilices Joblib a menos de que no tengas otra alternativa.

## Skops

Recientemente surgió una nueva biblioteca llamada skops, cuyo objetivo es ayudar a guardar modelos de machine learning y ponerlos en producción. 

Esta biblioteca forma parte de scikit-learn, sino que es 

Es sencilla de utilizar:

In [ ]:
import skops.io as sio

sio.dump(model, "model.sio")

Y para cargarlo:

In [ ]:
skopted_model = sio.load("model.sio")

Revisa el tipo y haz algunas predicciones para que veas que si funciona:

In [ ]:
type(skopted_model)

## Sobre las extensiones

En realidad no importan para nada las extensiones que le asignes a tu modelo, puesto que a final de cuentas para leerlos o escribirlos al disco deberás especificar la dirección completa hacia el archivo.

## Otras recomendaciones

Guardar los modelos es solo una parte de poner nuestros modelos en producción, otras cosas que debes hacer es 

Guardar los archivos es solo una parte de poner nuestros modelos en producción. Otras cosas que debes hacer son:

 1. <b>Mantener versiones consistentes de las bibliotecas</b>: Asegurarte de utilizar las mismas versiones, o algunas compatibles de scikit-learn y sus dependencias al guardar y cargar los modelos. Esto garantiza la compatibilidad y un comportamiento consistente del modelo.

 1. <b>Incluir pasos de preprocesamiento</b>: Guardar los transformadores de preprocesamiento entrenados junto con el modelo. Puedes lograr esto mediante la creación de un pipeline que incluya todos los pasos y guardar todo el pipeline.

 1. <b>Documentar tu modelo</b>: Considera documentar el propósito del modelo, las métricas de rendimiento, el conjunto de datos utilizado para el entrenamiento, featur engineering y cualquier otra información relevante. Esta documentación te ayudará a ti y a otros a comprender el contexto, las limitaciones y los casos de uso del modelo.

 1. <b>Usar un sistema de control de versiones</b>: Almacena tus modelos guardados y los archivos asociados (por ejemplo, scripts de preprocesamiento de datos, archivos de configuración y documentación) en un sistema de control de versiones como Git. 

 1. <b>Realizar copias de seguridad de tus modelos</b>: Asegúrate de que tus modelos guardados estén respaldados en un sistema de almacenamiento seguro y confiable. Esto podría implicar guardar modelos en almacenamiento en la nube o utilizar una solución de copia de seguridad dedicada.

 1. <b>Asegúrate que el modelo funciona después de guardarlo y abrirlo</b>: Después de guardar un modelo, prueba cargarlo y realizar predicciones para asegurarte de que el proceso de serialización y deserialización funcione según lo esperado. 

Y pues eso es todo, espero estos consejos te sirvan para poner tu modelo en producción de forma exitosa.